In [0]:
# Load Libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, make_scorer

In [15]:
!ls

!git clone https://github.com/papasanimohansrinivas/Data-Sets.git

datalab
Cloning into 'Data-Sets'...
remote: Counting objects: 9, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), done.


In [19]:
!cd Data-Sets
# !rm -r Data-Sets
!ls

# print('\n'*2)
# !ls

# print('\n'*2)


!unzip f2c2f440-8-dataset_he.zip

# print('\n'*2)

# !ls


/bin/sh: 1: cd: can't cd to Data-Sets
f2c2f440-8-dataset_he.zip  README.md  transformed_files.zip
Archive:  f2c2f440-8-dataset_he.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [18]:
# !ls
# cd Data-Sets

/content/Data-Sets


In [0]:
# load data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [75]:
train.head(8)

train.shape

(38932, 5)

In [0]:
# function to clean data
# import nltk

# nltk.download()
# stops = set(stopwords.words("english"))

stops = []

def cleanData(text, lowercase = False, remove_stops = False, stemming = False):
    txt = str(text)
    txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    txt = re.sub(r'\n',r' ',txt)
    
    if lowercase:
        txt = " ".join([w.lower() for w in txt.split()])
        
    if remove_stops:
        txt = " ".join([w for w in txt.split() if w not in stops])
    
    if stemming:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in txt.split()])

    return txt

In [0]:
## join data
test['Is_Response'] = np.nan
alldata = pd.concat([train, test]).reset_index(drop=True)

In [0]:
# clean description
alldata['Description'] = alldata['Description'].map(lambda x: cleanData(x, lowercase=True, remove_stops=False, stemming=False))

In [0]:
# initialise the functions - we'll create separate models for each type.
countvec = CountVectorizer(analyzer='word', ngram_range = (1,1), min_df=150, max_features=500)
tfidfvec = TfidfVectorizer(analyzer='word', ngram_range = (1,1), min_df = 150, max_features=500)

In [0]:
# create features
bagofwords = countvec.fit_transform(alldata['Description'])
tfidfdata = tfidfvec.fit_transform(alldata['Description'])

### just cehecking shape of bagofwords

In [28]:
bagofwords.shape

(68336, 500)

In [0]:
# label encode categorical features in data given
cols = ['Browser_Used','Device_Used']

for x in cols:
    lbl = LabelEncoder()
    alldata[x] = lbl.fit_transform(alldata[x])

In [0]:
# create dataframe for features
bow_df = pd.DataFrame(bagofwords.todense())
tfidf_df = pd.DataFrame(tfidfdata.todense())

In [0]:
# set column names
bow_df.columns = ['col'+ str(x) for x in bow_df.columns]
tfidf_df.columns = ['col' + str(x) for x in tfidf_df.columns]

In [0]:
# create separate data frame for bag of words and tf-idf

bow_df_train = bow_df[:len(train)]
bow_df_test = bow_df[len(train):]

tfid_df_train = tfidf_df[:len(train)]
tfid_df_test = tfidf_df[len(train):]

In [0]:
# split the merged data file into train and test respectively
train_feats = alldata[~pd.isnull(alldata.Is_Response)]
test_feats = alldata[pd.isnull(alldata.Is_Response)]



In [124]:
### set target variable

train_feats['Is_Response'] = [1 if x == 'happy' else 0 for x in train_feats['Is_Response']]

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
# merge count (bag of word) features into train
train_feats1 = pd.concat([train_feats[cols], bow_df_train], axis = 1)
test_feats1 = pd.concat([test_feats[cols], bow_df_test], axis=1)

test_feats1.reset_index(drop=True, inplace=True)

In [0]:
# merge into a new data frame with tf-idf features
train_feats2 = pd.concat([train_feats[cols], tfid_df_train], axis=1)
test_feats2 = pd.concat([test_feats[cols], tfid_df_test], axis=1)

In [127]:
# train_feats2.shape

targets = train_feats['Is_Response']

train_feats1.shape

'Is_Response' in list(test_feats1.columns)

False

### Keras Models Neural Network

In [0]:
# https://keras.io/

!pip install -q keras

import keras

In [0]:
from keras.models import Sequential

from keras.layers import Dense, Activation

model = Sequential()

In [157]:
# Adding the input layer and the first hidden layer
model.add(Dense(400, init = 'uniform', activation = 'relu', input_dim = 502))
# Adding the second hidden layer
model.add(Dense(20, init = 'uniform', activation = 'relu'))
# Adding the output layer
model.add(Dense(1, init = 'uniform', activation = 'sigmoid'))

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(400, activation="relu", kernel_initializer="uniform", input_dim=502)`
  """Entry point for launching an IPython kernel.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  """


In [0]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [159]:
model.fit(train_feats1,targets, batch_size = 100, nb_epoch = 80)

Epoch 1/80
38932/38932 [==============================] - 3s 74us/step - loss: 0.3759 - acc: 0.8390
Epoch 2/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.3161 - acc: 0.8664
Epoch 3/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.2960 - acc: 0.8733
Epoch 4/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.2707 - acc: 0.8855
Epoch 5/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.2360 - acc: 0.9035
Epoch 6/80
31400/38932 [=======================>......] - ETA: 0s - loss: 0.1879 - acc: 0.9256

38932/38932 [==============================] - 3s 68us/step - loss: 0.1876 - acc: 0.9257
Epoch 7/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.1343 - acc: 0.9521
Epoch 8/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0866 - acc: 0.9717
Epoch 9/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0534 - acc: 0.9841
Epoch 10/80
38932/38932 [==============================] - 3s 69us/step - loss: 0.0342 - acc: 0.9902
Epoch 11/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0261 - acc: 0.9923
Epoch 12/80
  100/38932 [..............................] - ETA: 3s - loss: 0.0371 - acc: 0.9900

38932/38932 [==============================] - 3s 68us/step - loss: 0.0162 - acc: 0.9958
Epoch 13/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0211 - acc: 0.9936
Epoch 14/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0076 - acc: 0.9980
Epoch 15/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0196 - acc: 0.9934
Epoch 16/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0238 - acc: 0.9909
Epoch 17/80
34000/38932 [=========================>....] - ETA: 0s - loss: 0.0125 - acc: 0.9962

38932/38932 [==============================] - 3s 68us/step - loss: 0.0139 - acc: 0.9957
Epoch 18/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0074 - acc: 0.9979
Epoch 19/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0031 - acc: 0.9992
Epoch 20/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0031 - acc: 0.9992
Epoch 21/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0204 - acc: 0.9929
Epoch 22/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0193 - acc: 0.9935
Epoch 23/80
  100/38932 [..............................] - ETA: 3s - loss: 0.0035 - acc: 1.0000

38932/38932 [==============================] - 3s 68us/step - loss: 0.0108 - acc: 0.9963
Epoch 24/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0092 - acc: 0.9974
Epoch 25/80
38932/38932 [==============================] - 3s 67us/step - loss: 0.0047 - acc: 0.9987
Epoch 26/80
38932/38932 [==============================] - 3s 66us/step - loss: 0.0056 - acc: 0.9983
Epoch 27/80
38932/38932 [==============================] - 3s 68us/step - loss: 0.0147 - acc: 0.9950
Epoch 28/80
35200/38932 [==========================>...] - ETA: 0s - loss: 0.0089 - acc: 0.9971

...

38932/38932 [==============================] - 3s 68us/step - loss: 4.1015e-06 - acc: 1.0000
Epoch 61/80
38932/38932 [==============================] - 3s 68us/step - loss: 3.4137e-06 - acc: 1.0000
Epoch 62/80
38932/38932 [==============================] - 3s 67us/step - loss: 2.8467e-06 - acc: 1.0000
Epoch 63/80
38932/38932 [==============================] - 3s 68us/step - loss: 2.3792e-06 - acc: 1.0000
Epoch 64/80
38932/38932 [==============================] - 3s 68us/step - loss: 1.9783e-06 - acc: 1.0000
Epoch 65/80
34700/38932 [=========================>....] - ETA: 0s - loss: 1.6382e-06 - acc: 1.0000

38932/38932 [==============================] - 3s 68us/step - loss: 1.6473e-06 - acc: 1.0000
Epoch 66/80
38932/38932 [==============================] - 3s 67us/step - loss: 1.3839e-06 - acc: 1.0000
Epoch 67/80
38932/38932 [==============================] - 3s 68us/step - loss: 1.1543e-06 - acc: 1.0000
Epoch 68/80
38932/38932 [==============================] - 3s 67us/step - loss: 9.6506e-07 - acc: 1.0000
Epoch 69/80
38932/38932 [==============================] - 3s 67us/step - loss: 8.2419e-07 - acc: 1.0000
Epoch 70/80
35000/38932 [=========================>....] - ETA: 0s - loss: 6.9130e-07 - acc: 1.0000

38932/38932 [==============================] - 3s 68us/step - loss: 6.9075e-07 - acc: 1.0000
Epoch 71/80
38932/38932 [==============================] - 3s 67us/step - loss: 5.9089e-07 - acc: 1.0000
Epoch 72/80
38932/38932 [==============================] - 3s 68us/step - loss: 5.0560e-07 - acc: 1.0000
Epoch 73/80
38932/38932 [==============================] - 3s 68us/step - loss: 4.3450e-07 - acc: 1.0000
Epoch 74/80
38932/38932 [==============================] - 3s 68us/step - loss: 3.7657e-07 - acc: 1.0000
Epoch 75/80
34600/38932 [=========================>....] - ETA: 0s - loss: 3.3313e-07 - acc: 1.0000

38932/38932 [==============================] - 3s 68us/step - loss: 3.2885e-07 - acc: 1.0000
Epoch 76/80
38932/38932 [==============================] - 3s 68us/step - loss: 2.9062e-07 - acc: 1.0000
Epoch 77/80
38932/38932 [==============================] - 3s 68us/step - loss: 2.5963e-07 - acc: 1.0000
Epoch 78/80
38932/38932 [==============================] - 3s 68us/step - loss: 2.3066e-07 - acc: 1.0000
Epoch 79/80
38932/38932 [==============================] - 3s 68us/step - loss: 2.0967e-07 - acc: 1.0000
Epoch 80/80
34900/38932 [=========================>....] - ETA: 0s - loss: 1.9213e-07 - acc: 1.0000

38932/38932 [==============================] - 3s 68us/step - loss: 1.9153e-07 - acc: 1.0000


In [0]:
y_pred = model.predict(test_feats1)
y_pred = (y_pred > 0.5)

In [0]:
# from sklearn.metrics import confusion_matrix
# cm = confusion_matrix(y_test, y_pred)

In [192]:
y_pred

array([[False],
       [ True],
       [False],
       ...,
       [ True],
       [ True],
       [ True]])

In [0]:
from pandas import DataFrame

test.User_ID.shape
y_pred = [f[0] for f in y_pred]
answers  = DataFrame({'User_ID':test['User_ID'],'Is_Response':y_pred})

In [0]:
answers.to_csv('neural_networks.csv',index=False)

In [195]:
!ls

f2c2f440-8-dataset_he.zip  sample_submission.csv  transformed_files.zip
neural_networks.csv	   test.csv
README.md		   train.csv


In [0]:
from google.colab import files


In [0]:
files.download('neural_networks.csv')

In [199]:
answers.head()

,Is_Response,User_ID
0,False,id80132
1,True,id80133
2,False,id80134
3,False,id80135
4,True,id80136


In [0]:
answers=answers[['User_ID','Is_Response']]

In [201]:
answers.head()

,User_ID,Is_Response
0,id80132,False
1,id80133,True
2,id80134,False
3,id80135,False
4,id80136,True


In [0]:
answers.to_csv('neural_networks.csv',index=False)

In [203]:
!ls

f2c2f440-8-dataset_he.zip  sample_submission.csv  transformed_files.zip
neural_networks.csv	   test.csv
README.md		   train.csv


In [0]:
files.download('neural_networks.csv')